In [ ]:
# LEGEND specific imports
import glob
from pathlib import Path

# MISC
import matplotlib.pyplot as plt
import numpy as np
from dspeed import build_dsp

# P-ONE fastDAQ decoder
from eng_reader import EngFormatReader
from lgdo import ArrayOfEqualSizedArrays, Table, WaveformTable, lh5

In [ ]:
# Just some paths
DAQ_PATH = "../data/gain_cal/daq/"
RAW_PATH = "../data/gain_cal/raw/"
DSP_PATH = "../data/gain_cal/dsp/"

# Hello Waveforms!

In [ ]:
%matplotlib ipympl

In [ ]:
help(ArrayOfEqualSizedArrays)

In [ ]:
f = open(DAQ_PATH + "gain_cal_95.txt")
r = EngFormatReader(f)

plt.figure()
for i in range(50):
    readout_test = r.read_next_trigger()
    if r.eof:
        break
    plt.plot(readout_test.waveforms[2][1:])
plt.show()

# Write P-One txt data to LEGEND raw format

We call the decoder on some MSU testdata
The data contains two relevant channels: channel 2 and channel 13

In [ ]:
f = open(DAQ_PATH + "gain_cal_95.txt")

r = EngFormatReader(f)
signal = {
    "ch002": [],
    "ch013": [],
}  # init a list for each channel to which we will write the waveforms
while True:
    readout_test = r.read_next_trigger()
    if r.eof:
        break
    # append the list of samples from each waveform to the list of waveforms
    # --> not very elegant, but does the trick for testing
    signal["ch002"].append(readout_test.waveforms[2])
    signal["ch013"].append(readout_test.waveforms[13])

Now we convert our waveforms into LEGEND data type and write it out to disk

In [ ]:
f_raw = RAW_PATH + "pone_meets_legend_gain_95_raw.lh5"
Path(f_raw).parent.mkdir(parents=True, exist_ok=True)
for k in signal:
    # lets convert the list of list into a rectangular structure (fingers crossed all entries have the same length)
    a = ArrayOfEqualSizedArrays(nda=np.array(signal[k], dtype=np.uint16))

    # Lets utilize one of the LEGEND specific data objects.
    # WaveformTable takes a rectangular array or a VectorOfVector (in case we have waveforms with different lengths)
    # and assigns sampling rate (here 4.8 ns), t_0 offset and units
    b = WaveformTable(values=a, dt=4.8, dt_units="ns", t0=0, t0_units="ns")

    # add everything into a lh5 structure and write it to disk
    table = Table(size=len(b))
    table.add_field("waveform", b, True)
    lh5.write(table, name="raw", group=k, lh5_file=f_raw)

Lets look what we have done.
the show command allows to view the files structure.
Hopefully we set it up in a way that it is understandable for anybody seeing this

In [ ]:
f_raw = RAW_PATH + "pone_meets_legend_gain_95_raw.lh5"
lh5.show(f_raw, attrs=True)

In [ ]:
tbl = lh5.read("ch002/raw", f_raw)
tbl2 = lh5.read("ch013/raw", f_raw)

In [ ]:
tbl.waveform.dt

In [ ]:
len(tbl), len(tbl2)

In [ ]:
plt.figure()
for wf in tbl.waveform.values.nda[:10]:
    plt.plot(wf[1:])
plt.ylim([2000, 2100])
plt.show()

In [ ]:
plt.figure()

plt.plot(tbl.waveform.values.nda[6])
plt.ylim([2040, 2100])
plt.show()

In [ ]:
plt.figure()
for wf in tbl2.waveform.values.nda[:100]:
    plt.plot(wf)
plt.show()

In [ ]:
tbl2.waveform.values.nda.shape

In [ ]:
len(tbl2.waveform.values.nda[0])

# RAW to DSP, signal processing on the fly

Ok. Now comes the fun part: Processing our waveforms!
First we define a processing configuration and outputs.
Here we do
- hermetic spline upsampling of the waveform to 1 ns
- apply an average fillter to remove some noise
- find the baseline by applying a linear fit to the first four samples of the waveform
- substracting the baseline from the waveform
- finding global extrema
- Building a +1 15 ns coincidence window around the maximum
- integrate the waveform insite the coincidence window
- Picking of the maximum of the integral --> charge

In [ ]:
from dspeed.vis import WaveformBrowser

In [ ]:
config = {
    "outputs": [
        "tp_max",
        "wf_max",
        "charge",
    ],
    "processors": {
        "wf_up": {
            "function": "interpolating_upsampler",
            "module": "dspeed.processors",
            "args": ["waveform", "'h'", "wf_up(len(waveform)*9.6, period=waveform.period/9.6)"],
            "unit": "ADC",
        },
        "wf_av": {
            "function": "moving_window_multi",
            "module": "dspeed.processors",
            "args": ["wf_up", "8", "4", "0", "wf_av"],
            "unit": "ADC/sample",
        },
        "bl_mean , bl_std, bl_slope, bl_intercept": {
            "function": "linear_slope_fit",
            "module": "dspeed.processors",
            "args": ["wf_up[1:50]", "bl_mean", "bl_std", "bl_slope", "bl_intercept"],
            "unit": ["ADC", "ADC", "ADC", "ADC"],
        },
        "wf_blsub": {
            "function": "subtract",
            "module": "numpy",
            "args": ["wf_av", "bl_mean", "wf_blsub"],
            "unit": "ADC",
        },
        "wf_rc_cr2": {
            "function": "rc_cr2",
            "module": "dspeed.processors",
            "args": ["wf_blsub", "20*ns", "wf_rc_cr2"],
            "unit": "ADC",
        },
        "tp_min, tp_max, wf_min, wf_max": {
            "function": "min_max",
            "module": "dspeed.processors",
            "args": ["wf_blsub", "tp_min", "tp_max", "wf_min", "wf_max"],
            "unit": ["ns", "ns", "ADC", "ADC"],
        },
        "wf_coin": {
            "function": "windower",
            "module": "dspeed.processors",
            "args": [
                "wf_blsub",
                "tp_max-15*ns",
                "wf_coin(shape=round(30*ns/wf_blsub.period), period=wf_blsub.period, offset=(tp_max-15*ns)/wf_blsub.period)",
            ],
            "unit": ["ADC"],
        },
        "wf_cum_coin": {
            "function": "cumsum",
            "module": "numpy",
            "args": ["wf_coin", 1, "None", "wf_cum_coin"],
            "kwargs": {"signature": "(n),(),()->(n)", "types": ["fii->f"]},
            "unit": "ADC",
        },
        "tp_min_int, tp_max_int, wf_min_int, charge": {
            "function": "min_max",
            "module": "dspeed.processors",
            "args": ["wf_cum_coin", "tp_min_int", "tp_max_int", "wf_min_int", "charge"],
            "unit": ["ns", "ns", "ADC", "ADC"],
        },
        "wf_cumsum": {
            "function": "cumsum",
            "module": "numpy",
            "args": ["wf_blsub", 1, "None", "wf_cumsum"],
            "kwargs": {"signature": "(n),(),()->(n)", "types": ["fii->f"]},
            "unit": "ADC",
        },
    },
}

LEGENDs signal processing package (dspeed) includes a WaveformBrowser with which we can apply processors from our above configuration on the fly, and look at the output.
Lets plot on of the upsampled, baseline substracted waveforms and mark the maximum 

In [ ]:
entry_no = 6
browser = WaveformBrowser(
    RAW_PATH + "pone_meets_legend_gain_95_raw.lh5",
    "ch002/raw",
    dsp_config=config,
    lines=["wf_blsub", "tp_max", "wf_rc_cr2"],
    # styles=[
    #     {"color": ["r"], "ls": ["dotted"]},
    # {"color": ["g"], "ls": ["dotted"]},
    # {"color": ["skyblue"] , "ls": ["dotted"]},
    # ],
    # y_lim=(-10, 100),
)
browser.draw_entry(entry_no)

Now that we are happy with our processing chain. We need only one more line to create our DSP level

In [ ]:
f_raw = glob.glob(RAW_PATH + "*")

for f in f_raw:
    f_dsp = f.replace("raw", "dsp")
    build_dsp(
        f_raw=f,
        f_dsp=f_dsp,
        dsp_config=config,
        write_mode="o",
    )

Lets have a look into our created dsp file!

In [ ]:
f_dsp = DSP_PATH + "pone_meets_legend_gain_95_dsp.lh5"
lh5.show(f_dsp, attrs=True)

Lets also plot the p.e. spectra.

In [ ]:
f_dsp = glob.glob(DSP_PATH + "*.lh5")

fig, ax = plt.subplots(3)
for f in f_dsp:
    voltage = f.split("_")[-2]

    trig_dsp = lh5.read_as("ch013/dsp", f, "ak")
    pmt_dsp = lh5.read_as("ch002/dsp", f, "ak")

    ax[0].hist(
        pmt_dsp.charge, bins=np.linspace(0, 500, 125), histtype="step", label=f"{voltage} V"
    )
    ax[1].hist(
        pmt_dsp.tp_max, bins=np.linspace(0, 150, 150), histtype="step", label=f"{voltage} V"
    )
    ax[2].hist(
        trig_dsp.tp_max, bins=np.linspace(0, 150, 150), histtype="step", label=f"{voltage} V"
    )
ax[0].set_yscale("log")
ax[2].set_yscale("log")
ax[0].legend()
# ax[0].set_xlim(0,100)

# DSP to HIT: Applying functions to DSP data (i.e. calibration curves)